In [13]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
import tensorflow.keras.backend as K
import numpy as np
from tensorflow import keras
import time
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import RandomFourierFeatures
import pandas as pd
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from __future__ import division
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [14]:
dt = pd.read_csv('SVMData/test_feat.csv', index_col=0)

In [15]:
dataset = dt.sample(10000)
dataset

,all#count,session#count,seek_video#num,play_video#num,pause_video#num,stop_video#num,load_video#num,problem_get#num,problem_check#num,problem_save#num,...,truth,username,course_id,age,gender,education,user_enroll_num,course_enroll_num,cluster_label,course_category
enroll_id,,,,,,,,,,,,,,,,,,,,,
235325,-0.187565,-0.187565,-0.243043,-0.269383,-0.258490,-0.059298,-0.338163,-0.042412,-0.305844,-0.202161,...,1,1458804,course-v1:Stanford+Econ1+_,-0.599969,0,0,0.256741,0.872874,2,0
394058,-0.227150,-0.227150,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,1,1525032,course-v1:TsinghuaX+00510663X+2015_T2,-0.599969,0,0,-0.271904,0.077523,2,0
120154,-0.103446,-0.103446,-0.243043,-0.118926,-0.308520,-0.059298,0.875679,-0.042412,-0.305844,-0.202161,...,0,86960,TsinghuaX/80511503X/2015_T2,1.432103,1,1,0.080526,2.094772,3,0
427575,-0.229624,-0.229624,-0.243043,-0.296739,-0.333535,-0.059298,-0.445267,-0.042412,-0.305844,-0.202161,...,1,101671,TsinghuaX/34100325X/2015_T1,1.359529,1,3,0.080526,-0.112467,2,0
461974,-0.213542,-0.213542,-0.243043,-0.296739,-0.296012,-0.059298,-0.373865,-0.042412,-0.305844,-0.202161,...,1,580607,course-v1:MITx+6_00_1x+2015_T2,-0.599969,0,0,-0.360012,-0.394658,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337886,-0.152928,-0.152928,-0.115253,-0.187316,-0.170937,-0.041827,-0.123956,-0.039505,-0.305844,-0.202161,...,1,2789297,course-v1:TsinghuaX+80512073X+2016_T1,-0.599969,2,0,-0.360012,2.580924,2,0
272409,0.103138,0.103138,-0.093955,0.277735,0.066706,-0.035474,0.625770,0.004102,1.392301,-0.013096,...,1,374520,course-v1:TsinghuaX+60240013X+2015_T2,-0.599969,1,1,-0.095689,-0.504555,0,0
267777,-0.223438,-0.223438,-0.243043,-0.296739,-0.321027,-0.059298,-0.445267,-0.042412,-0.305844,-0.202161,...,1,3206736,course-v1:MITx+15_390x_2016T1+2016_TS,-0.599969,0,0,-0.360012,-0.748562,2,0


In [16]:
dataset = pd.get_dummies(dataset, columns=['gender'])
dataset.head()

,all#count,session#count,seek_video#num,play_video#num,pause_video#num,stop_video#num,load_video#num,problem_get#num,problem_check#num,problem_save#num,...,course_id,age,education,user_enroll_num,course_enroll_num,cluster_label,course_category,gender_0,gender_1,gender_2
enroll_id,,,,,,,,,,,,,,,,,,,,,
235325,-0.187565,-0.187565,-0.243043,-0.269383,-0.258490,-0.059298,-0.338163,-0.042412,-0.305844,-0.202161,...,course-v1:Stanford+Econ1+_,-0.599969,0,0.256741,0.872874,2,0,1,0,0
394058,-0.227150,-0.227150,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,course-v1:TsinghuaX+00510663X+2015_T2,-0.599969,0,-0.271904,0.077523,2,0,1,0,0
120154,-0.103446,-0.103446,-0.243043,-0.118926,-0.308520,-0.059298,0.875679,-0.042412,-0.305844,-0.202161,...,TsinghuaX/80511503X/2015_T2,1.432103,1,0.080526,2.094772,3,0,0,1,0
427575,-0.229624,-0.229624,-0.243043,-0.296739,-0.333535,-0.059298,-0.445267,-0.042412,-0.305844,-0.202161,...,TsinghuaX/34100325X/2015_T1,1.359529,3,0.080526,-0.112467,2,0,0,1,0
461974,-0.213542,-0.213542,-0.243043,-0.296739,-0.296012,-0.059298,-0.373865,-0.042412,-0.305844,-0.202161,...,course-v1:MITx+6_00_1x+2015_T2,-0.599969,0,-0.360012,-0.394658,2,0,1,0,0


In [17]:
dataset = pd.get_dummies(dataset, columns=['education'])
dataset.head()

,all#count,session#count,seek_video#num,play_video#num,pause_video#num,stop_video#num,load_video#num,problem_get#num,problem_check#num,problem_save#num,...,gender_1,gender_2,education_0,education_1,education_2,education_3,education_4,education_5,education_6,education_7
enroll_id,,,,,,,,,,,,,,,,,,,,,
235325,-0.187565,-0.187565,-0.243043,-0.269383,-0.258490,-0.059298,-0.338163,-0.042412,-0.305844,-0.202161,...,0,0,1,0,0,0,0,0,0,0
394058,-0.227150,-0.227150,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,0,0,1,0,0,0,0,0,0,0
120154,-0.103446,-0.103446,-0.243043,-0.118926,-0.308520,-0.059298,0.875679,-0.042412,-0.305844,-0.202161,...,1,0,0,1,0,0,0,0,0,0
427575,-0.229624,-0.229624,-0.243043,-0.296739,-0.333535,-0.059298,-0.445267,-0.042412,-0.305844,-0.202161,...,1,0,0,0,0,1,0,0,0,0
461974,-0.213542,-0.213542,-0.243043,-0.296739,-0.296012,-0.059298,-0.373865,-0.042412,-0.305844,-0.202161,...,0,0,1,0,0,0,0,0,0,0


In [18]:
target = dataset.pop('truth')
trash= dataset.pop('course_id')
trash2= dataset.pop('course_category')
trash3 = dataset.pop('cluster_label')
dataset.head()

,all#count,session#count,seek_video#num,play_video#num,pause_video#num,stop_video#num,load_video#num,problem_get#num,problem_check#num,problem_save#num,...,gender_1,gender_2,education_0,education_1,education_2,education_3,education_4,education_5,education_6,education_7
enroll_id,,,,,,,,,,,,,,,,,,,,,
235325,-0.187565,-0.187565,-0.243043,-0.269383,-0.258490,-0.059298,-0.338163,-0.042412,-0.305844,-0.202161,...,0,0,1,0,0,0,0,0,0,0
394058,-0.227150,-0.227150,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,0,0,1,0,0,0,0,0,0,0
120154,-0.103446,-0.103446,-0.243043,-0.118926,-0.308520,-0.059298,0.875679,-0.042412,-0.305844,-0.202161,...,1,0,0,1,0,0,0,0,0,0
427575,-0.229624,-0.229624,-0.243043,-0.296739,-0.333535,-0.059298,-0.445267,-0.042412,-0.305844,-0.202161,...,1,0,0,0,0,1,0,0,0,0
461974,-0.213542,-0.213542,-0.243043,-0.296739,-0.296012,-0.059298,-0.373865,-0.042412,-0.305844,-0.202161,...,0,0,1,0,0,0,0,0,0,0


In [19]:
# Spliting into 80% for training set and 20% for testing set so we can see our accuracy
X_train, x_test, Y_train, y_test = train_test_split(dataset, target, test_size=0.2, random_state=0)

In [20]:
from sklearn.svm import LinearSVC

clf_svc = LinearSVC(penalty="l2", dual=False, tol=1e-3)
clf_svc.fit(X_train, Y_train)

LinearSVC(dual=False, tol=0.001)

In [21]:
predicted = clf_svc.predict(x_test)
predicted

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [22]:
from sklearn.metrics import accuracy_score

acc_svm = accuracy_score(y_test, predicted)

In [23]:
acc_svm

0.754